In [30]:
import sys
import glob
import yaml
import graphviz as gv
import re

**Функция визуализации**

In [31]:
styles = {
    "graph": {
        "label": "Network Map",
        "fontsize": "16",
        "fontcolor": "white",
        "bgcolor": "#3F3F3F",
        "rankdir": "BT",
    },
    "nodes": {
        "fontname": "Helvetica",
        "shape": "box",
        "fontcolor": "white",
        "color": "#006699",
        "style": "filled",
        "fillcolor": "#006699",
        "margin": "0.4",
    },
    "edges": {
        "style": "dashed",
        "color": "green",
        "arrowhead": "open",
        "fontname": "Courier",
        "fontsize": "14",
        "fontcolor": "white",
    },
}


def apply_styles(graph, styles):
    graph.graph_attr.update(("graph" in styles and styles["graph"]) or {})
    graph.node_attr.update(("nodes" in styles and styles["nodes"]) or {})
    graph.edge_attr.update(("edges" in styles and styles["edges"]) or {})
    return graph


def draw_topology(topology_dict, out_filename="img/topology", style_dict=styles):
    """
    topology_dict - словарь с описанием топологии

    Пример словаря topology_dict:
        {('R4', 'Eth0/1'): ('R5', 'Eth0/1'),
         ('R4', 'Eth0/2'): ('R6', 'Eth0/0')}

    соответствует топологии:
    [ R5 ]-Eth0/1 --- Eth0/1-[ R4 ]-Eth0/2---Eth0/0-[ R6 ]

    Функция генерирует топологию, в формате svg.
    И записывает файл topology.svg в каталог img.
    """
    nodes = set(
        [item[0] for item in list(topology_dict.keys()) + list(topology_dict.values())]
    )

    graph = gv.Graph(format="svg")

    for node in nodes:
        graph.node(node)

    for key, value in topology_dict.items():
        head, t_label = key
        tail, h_label = value
        graph.edge(head, tail, headlabel=h_label, taillabel=t_label, label=" " * 12)

    graph = apply_styles(graph, style_dict)
    filename = graph.render(filename=out_filename)
    print("Topology saved in", filename)

Словарь

In [32]:
def parse_sh_cdp_neighbors(command_output):
    regex = re.compile(
        r"(?P<r_dev>\w+)  +(?P<l_intf>\S+ \S+)"
        r"  +\d+  +[\w ]+  +\S+ +(?P<r_intf>\S+ \S+)"
    )
    connect_dict = {}
    l_dev = re.search(r"(\S+)[>#]", command_output).group(1)
    connect_dict[l_dev] = {}
    for match in regex.finditer(command_output):
        r_dev, l_intf, r_intf = match.group("r_dev", "l_intf", "r_intf")
        connect_dict[l_dev][l_intf] = {r_dev: r_intf}
    return connect_dict

In [33]:
with open("sh_cdp_n_r3.txt") as f:
  test = parse_sh_cdp_neighbors(f.read())
  test

In [34]:
def generate_topology_from_cdp(list_of_files, save_to_filename=None):
    topology = {}
    for filename in list_of_files:
        with open(filename) as f:
            topology.update(parse_sh_cdp_neighbors(f.read()))
    if save_to_filename:
        with open(save_to_filename, "w") as f_out:
            yaml.dump(topology, f_out, default_flow_style=False)
    return topology

In [35]:
f_list = [
"sh_cdp_n_sw1.txt",
"sh_cdp_n_r1.txt",
"sh_cdp_n_r2.txt",
"sh_cdp_n_r3.txt",
"sh_cdp_n_r4.txt",
"sh_cdp_n_r5.txt",
"sh_cdp_n_r6.txt",
]

In [36]:
f_list = glob.glob("sh_cdp_n_*")
generate_topology_from_cdp(f_list, save_to_filename="topology.yaml")

{'R6': {'Eth 0/1': {'R2': 'Eth 0/2'}},
 'R3': {'Eth 0/0': {'SW1': 'Eth 0/3'}},
 'R1': {'Eth 0/0': {'SW1': 'Eth 0/1'}},
 'SW1': {'Eth 0/1': {'R1': 'Eth 0/0'},
  'Eth 0/2': {'R2': 'Eth 0/0'},
  'Eth 0/3': {'R3': 'Eth 0/0'},
  'Eth 0/4': {'R4': 'Eth 0/0'}},
 'R2': {'Eth 0/0': {'SW1': 'Eth 0/2'},
  'Eth 0/1': {'R5': 'Eth 0/0'},
  'Eth 0/2': {'R6': 'Eth 0/1'}},
 'R5': {'Eth 0/0': {'R2': 'Eth 0/1'}, 'Eth 0/1': {'R4': 'Eth 0/1'}},
 'R4': {'Eth 0/0': {'SW1': 'Eth 0/4'}, 'Eth 0/1': {'R5': 'Eth 0/1'}}}

In [37]:
def transform_topology(topology_filename):
    with open(topology_filename) as f:
        raw_topology = yaml.safe_load(f)

    formatted_topology = {}
    for l_device, peer in raw_topology.items():
        for l_int, remote in peer.items():
            r_device, r_int = list(remote.items())[0]
            if not (r_device, r_int) in formatted_topology:
                formatted_topology[(l_device, l_int)] = (r_device, r_int)
    return formatted_topology

In [38]:
formatted_topology = transform_topology("topology.yaml")
draw_topology(formatted_topology)

Topology saved in img/topology.svg
